In [ ]:
import pandas as pd
import os
from src import *


In [ ]:

df = pd.read_parquet('.data/frequency_raw_combined.parquet')
progress = pd.read_parquet('.data/opensnp_progress.parquet')

def create_reduced_dataframe(df):
    if os.path.exists('.data/reduced_temp.parquet'):
        return pd.read_parquet('.data/reduced_temp.parquet')
    
    reduced = df[df['rsid'].str.startswith('rs') | df['rsid'].str.startswith('i')][['rsid','chromosome','alleles','count']]
    reduced['alleles'] = reduced['alleles'].str.replace('None','')
    reduced = reduced.groupby(['rsid','chromosome','alleles'])['count'].sum()
    reduced = reduced.reset_index(drop=False)
    reduced.to_parquet('.data/reduced_temp.parquet')
    return reduced

reduced = create_reduced_dataframe(df)

In [ ]:
reduced = reduced[reduced['alleles'] != '--']

In [ ]:
reduced[(reduced['rsid'] == 'rs63750767') & (reduced['chromosome'] == '1')]

In [ ]:
def find_source_for(values:dict):
    print('searching the combined parquet files for the value')
    subsets_with_value = []

    filelist = [x for x in os.listdir('.data/combined_subsets/') if x.endswith('.parquet')]
    print('Searching combined subsets')
    for x,_ in zip(filelist, trange(len(filelist) - 1)):
        fullpath = os.path.join('.data', 'combined_subsets', x)
        df = pd.read_parquet(fullpath)
        for k,v in values.items():
            df = df[df[k] == v].copy()
        if len(df) > 0:
            subsets_with_value.append(x)

    reader = GeneticDataToDataFrameConverter()

    files = progress.loc[progress['out_file'].isin(subsets_with_value) & (progress['errored'] == False), 'filename'].tolist()

    files_containing_data = []
    
    for file,_ in zip(files, trange(len(files) - 1)):
        pathname = os.path.join('.data','opensnp',file)
        df = reader.read_data(pathname, True)
        for k,v in values.items():
            df = df[df[k] == v].copy()
        if len(df) > 0:
            files_containing_data.append(file)

    return files_containing_data

result = find_source_for({'rsid':'rs63750767', 'chromosome':'1'})

In [ ]:
result